In [191]:
import re

In [192]:
corpus_name = 'en_ewt-ud-train.conllu'
with open(corpus_name) as f:
    corpus = []
    #corpus  = re.findall(r"^\d\t(.*)", data)
    for line in f.readlines():
        if re.match(r"^\d\t(.*)", line) != None:
            line = re.split(r'\t', line)
            corpus.append(line)
print('Corpus length (in words) = %d' % len(corpus))

Corpus length (in words) = 95250


Пусть первые 7000 слов будут обучающей выборкой, а остальные контрольной

In [193]:
test = corpus[7002:]
corpus = corpus[:7002]

##  HMM-tagger

In [209]:
tags = {}
prob_tags = {}
for token in range(len(corpus)):
    tag = corpus[token][4]
    counter = tags.get(tag)
    if counter == None:
        counter = 0
    tags.update({tag:counter+1})
for tag in tags:
    tag_count = tags.get(tag)
    count = 1 + tag_count/len(corpus)
    prob_tags.update({tag:count})

In [210]:
prob_tags

{'NNP': 1.115538417594973,
 'HYPH': 1.0074264495858327,
 ':': 1.0051413881748072,
 'JJ': 1.0609825764067409,
 'NNS': 1.0405598400457012,
 'VBD': 1.0438446158240502,
 '-LRB-': 1.0037132247929164,
 'DT': 1.1008283347614967,
 'NN': 1.101399600114253,
 'IN': 1.109397315052842,
 'MD': 1.0119965724078834,
 'VB': 1.0304198800342759,
 'PRP': 1.0408454727220795,
 'VBN': 1.0261353898886032,
 'CD': 1.0178520422736361,
 'VBG': 1.0172807769208798,
 'VBZ': 1.032419308768923,
 'VBP': 1.0215652670665525,
 'CC': 1.0231362467866323,
 'RB': 1.0549842902027993,
 'TO': 1.0119965724078834,
 ',': 1.0424164524421593,
 'PRP$': 1.009568694658669,
 '.': 1.0101399600114254,
 'WDT': 1.003284775778349,
 'RBR': 1.001856612396458,
 'NNPS': 1.0075692659240216,
 'EX': 1.0041416738074835,
 'JJS': 1.0019994287346472,
 'WP': 1.0044273064838618,
 '``': 1.0081405312767782,
 "''": 1.0012853470437018,
 'WRB': 1.003284775778349,
 'POS': 1.0077120822622108,
 'RBS': 1.0008568980291346,
 '-RRB-': 1.0021422450728363,
 'ADD': 1.004

In [224]:
tag_word_bag = {}
word_tag_bag = {}
for token in range(len(corpus)):
    pair = (corpus[token][4], corpus[token][1])
    counter = tag_word_bag.get(pair)
    if counter == None:
        counter = 0
    tag_word_bag.update({pair:counter+1})

for thing in tag_word_bag:
    tag = thing[0]
    word = thing[1]
    count = 1 + tag_word_bag.get(thing)/tags.get(tag)
    word_tag_bag.setdefault(word, {})[tag] = count

In [236]:
tag_bigram_ = {}
tag_bigram = {}
for thing in range(len(corpus)-1):
    pair = (corpus[thing][4], corpus[thing+1][4])
    count = tag_bigram_.get(pair)
    if count == None:
        count = 0
    tag_bigram_.update({pair:count+1})

for thing in tag_bigram_:
    tag_1 = thing[0]
    tag_2 = thing[1]
    count = 1 + tag_bigram_.get(thing)/len(tag_bigram_)
    tag_bigram.setdefault(tag_1, {})[tag_2] = count

tag_bigram

{'NNP': {'HYPH': 1.0355555555555556,
  ':': 1.0237037037037038,
  'NNP': 1.3111111111111111,
  '-LRB-': 1.0148148148148148,
  'IN': 1.0725925925925925,
  'VBZ': 1.1007407407407408,
  'DT': 1.031111111111111,
  'CC': 1.0503703703703704,
  'NNS': 1.0222222222222221,
  'VBD': 1.117037037037037,
  ',': 1.0888888888888888,
  'NN': 1.0518518518518518,
  'CD': 1.0281481481481483,
  'VBP': 1.0118518518518518,
  '``': 1.008888888888889,
  'RB': 1.04,
  'PRP': 1.0162962962962963,
  'POS': 1.068148148148148,
  '-RRB-': 1.0074074074074073,
  '.': 1.005925925925926,
  'MD': 1.0296296296296297,
  'TO': 1.002962962962963,
  'VBG': 1.0074074074074073,
  'JJR': 1.0014814814814814,
  'JJ': 1.0177777777777777,
  'NNPS': 1.0162962962962963,
  'WDT': 1.0014814814814814,
  'WP': 1.0014814814814814,
  'PRP$': 1.0014814814814814,
  "''": 1.002962962962963,
  'VB': 1.005925925925926,
  'WRB': 1.002962962962963},
 'HYPH': {'NNP': 1.0296296296296297,
  'PRP': 1.0044444444444445,
  'NN': 1.0148148148148148,
  'VB

In [242]:
tagged_seq = []
for word in range(len(test)):
    prob = []
    if test[word][0] == 0:
        prob_tags = [['SenInit', 1]]
    else:
        prev_tag = tagged_seq[word-1][1]
        prob_tags_ = tag_bigram.get(prev_tag)
        prob_tags = []
        for thing in prob_tags_:
            tag_prob = []
            tag_prob.append(thing)
            pro = prob_tags_.get(thing)
            tag_prob.append(pro)
        prob_tags.append(tag_prob)
prob_tags

IndexError: list index out of range